In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

/Users/kuoteng/.pyenv/versions/3.6.4/envs/NCKU-DSAI/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
DATA_SIZE = 60000
TRAIN_SIZE = 50000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
EPOCH_SIZE = 2
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '-',
 3: '0',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

# Data Generation

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')

Generating data...


In [8]:
while len(questions) < DATA_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    if len(questions) % 2 == 0:
        q = '{}-{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a - b)
    else:
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a + b)
    if q in seen:
        continue
    seen.add(q)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Total addition questions: 60000


In [9]:
print(questions[:5], expected[:5])

['1-254  ', '0+993  ', '3-24   ', '349+5  ', '51-11  '] ['-253', '993 ', '-21 ', '354 ', '40  ']


# Processing

In [10]:
print('Vectorization... (to the one-hot encoding)')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization... (to the one-hot encoding)


In [11]:
indices = np.arange(len(y))
np.random.shuffle(indices)
print(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:TRAIN_SIZE]
train_y = y[:TRAIN_SIZE]
test_x = x[TRAIN_SIZE:]
test_y = y[TRAIN_SIZE:]

print('Training Data:')
print(train_x.shape)
print(train_y.shape)

split_at = len(train_x) - len(train_x) // 10
print('split_at', split_at)
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)


[12147  6319 12809 ... 40602 36593 35611]
Training Data:
(50000, 7, 13)
(50000, 4, 13)
split_at 45000
Training Data:
(45000, 7, 13)
(45000, 4, 13)
Validation Data:
(5000, 7, 13)
(5000, 4, 13)
Testing Data:
(10000, 7, 13)
(10000, 4, 13)


In [12]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False False False
   False  True]
  [False  True False False False False False False False False False
   False False]
  [False False False False  True False False False False False False
   False False]
  [False False False False False False False False  True False False
   False False]
  [ True False False False False False False False False False False
   False False]
  [ True False False False False False False False False False False
   False False]
  [ True False False False False False False False False False False
   False False]]

 [[False False False False False False False False False False  True
   False False]
  [False  True False False False False False False False False False
   False False]
  [False False False False  True False False False False False False
   False False]
  [False False False False False False False False False False  True
   False False]
  [False False False False False False  True False Fals

# Build Model

In [13]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72704     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 13)             1677      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 13)             0         
Total params: 205,965
Trainable params: 205,965
Non-trainable params: 0
_________________________________________________________________


# Training

In [14]:
% time
for loop in range(50):
    print()
    print('-' * 50)
    print('Train Loop Num:', loop)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCH_SIZE,
              validation_data=(x_val, y_val),
              shuffle=True)
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs

--------------------------------------------------
Train Loop Num: 0
Train on 45000 samples, validate on 5000 samples
Epoch 1/2
45000/45000 [==============================] - 13s 290us/step - loss: 1.8898 - acc: 0.3317 - val_loss: 1.7031 - val_acc: 0.3752
Epoch 2/2
45000/45000 [==============================] - 12s 259us/step - loss: 1.6795 - acc: 0.3836 - val_loss: 1.6394 - val_acc: 0.3986
Q 7-383   T -376 ☒ -866
Q 87-762  T -675 ☒ -665
Q 9+969   T 978  ☒ 199 
Q 4+103   T 107  ☒ 33  
Q 553+248 T 801  ☒ 143 
Q 6+816   T 822  ☒ 165 
Q 44-581  T -537 ☒ -445
Q 922-844 T 78   ☒ 115 
Q 39-41   T -2   ☒ 31  
Q 4-908   T -904 ☒ -983

--------------------------------------------------
Train Loop Num: 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/2
45000/45000 [==============================] - 11s 245us/step - loss: 1.6114 - acc: 0.4065 - val_loss: 1.5613 - val_acc: 0.4245
Epoch 2/2
45000/45000 [=================

45000/45000 [==============================] - 11s 245us/step - loss: 0.6886 - acc: 0.7424 - val_loss: 0.7099 - val_acc: 0.7254
Epoch 2/2
45000/45000 [==============================] - 11s 240us/step - loss: 0.6481 - acc: 0.7585 - val_loss: 0.6605 - val_acc: 0.7440
Q 87-434  T -347 ☒ -354
Q 965-27  T 938  ☑ 938 
Q 95+4    T 99   ☑ 99  
Q 3-869   T -866 ☑ -866
Q 314-613 T -299 ☒ -399
Q 3+177   T 180  ☑ 180 
Q 620-33  T 587  ☒ 591 
Q 569+5   T 574  ☒ 564 
Q 883-93  T 790  ☒ 803 
Q 615+66  T 681  ☒ 671 

--------------------------------------------------
Train Loop Num: 12
Train on 45000 samples, validate on 5000 samples
Epoch 1/2
45000/45000 [==============================] - 11s 246us/step - loss: 0.6116 - acc: 0.7745 - val_loss: 0.6405 - val_acc: 0.7534
Epoch 2/2
45000/45000 [==============================] - 11s 242us/step - loss: 0.5761 - acc: 0.7882 - val_loss: 0.6116 - val_acc: 0.7661
Q 672+171 T 843  ☒ 863 
Q 585+57  T 642  ☑ 642 
Q 558-12  T 546  ☒ 555 
Q 65-27   T 38   ☑ 38  
Q 

45000/45000 [==============================] - 11s 254us/step - loss: 0.1613 - acc: 0.9509 - val_loss: 0.2341 - val_acc: 0.9133
Q 80-13   T 67   ☒ 77  
Q 33-245  T -212 ☑ -212
Q 16+96   T 112  ☒ 113 
Q 12+355  T 367  ☑ 367 
Q 545-41  T 504  ☒ 405 
Q 7-637   T -630 ☑ -630
Q 337+5   T 342  ☑ 342 
Q 806+17  T 823  ☑ 823 
Q 8-163   T -155 ☑ -155
Q 719-98  T 621  ☑ 621 

--------------------------------------------------
Train Loop Num: 23
Train on 45000 samples, validate on 5000 samples
Epoch 1/2
45000/45000 [==============================] - 12s 261us/step - loss: 0.1501 - acc: 0.9546 - val_loss: 0.2244 - val_acc: 0.9226
Epoch 2/2
45000/45000 [==============================] - 10s 231us/step - loss: 0.1534 - acc: 0.9521 - val_loss: 0.2148 - val_acc: 0.9228
Q 566+640 T 1206 ☒ 1106
Q 62-334  T -272 ☑ -272
Q 66+833  T 899  ☑ 899 
Q 6+175   T 181  ☑ 181 
Q 532+55  T 587  ☒ 597 
Q 127-727 T -600 ☒ -611
Q 905+75  T 980  ☑ 980 
Q 0-506   T -506 ☑ -506
Q 48+408  T 456  ☑ 456 
Q 439-61  T 378  ☑ 3

45000/45000 [==============================] - 10s 226us/step - loss: 0.0750 - acc: 0.9789 - val_loss: 0.2539 - val_acc: 0.9130
Epoch 2/2
45000/45000 [==============================] - 10s 226us/step - loss: 0.0784 - acc: 0.9770 - val_loss: 0.1390 - val_acc: 0.9501
Q 790-7   T 783  ☑ 783 
Q 874+34  T 908  ☑ 908 
Q 962-26  T 936  ☑ 936 
Q 657+83  T 740  ☑ 740 
Q 21-30   T -9   ☑ -9  
Q 461-24  T 437  ☑ 437 
Q 320+64  T 384  ☑ 384 
Q 0-396   T -396 ☑ -396
Q 886-719 T 167  ☑ 167 
Q 378+824 T 1202 ☒ 1102

--------------------------------------------------
Train Loop Num: 35
Train on 45000 samples, validate on 5000 samples
Epoch 1/2
45000/45000 [==============================] - 11s 242us/step - loss: 0.0481 - acc: 0.9895 - val_loss: 0.1301 - val_acc: 0.9524
Epoch 2/2
45000/45000 [==============================] - 12s 272us/step - loss: 0.0697 - acc: 0.9800 - val_loss: 0.1565 - val_acc: 0.9426
Q 81-19   T 62   ☑ 62  
Q 1+458   T 459  ☑ 459 
Q 466-4   T 462  ☑ 462 
Q 115-75  T 40   ☒ 31  
Q 

45000/45000 [==============================] - 13s 278us/step - loss: 0.0480 - acc: 0.9865 - val_loss: 0.1598 - val_acc: 0.9466
Q 3-860   T -857 ☑ -857
Q 53-84   T -31  ☑ -31 
Q 434+9   T 443  ☑ 443 
Q 0+355   T 355  ☑ 355 
Q 777+92  T 869  ☑ 869 
Q 18+86   T 104  ☑ 104 
Q 59+45   T 104  ☑ 104 
Q 827+478 T 1305 ☑ 1305
Q 435-7   T 428  ☑ 428 
Q 6+638   T 644  ☑ 644 

--------------------------------------------------
Train Loop Num: 46
Train on 45000 samples, validate on 5000 samples
Epoch 1/2
45000/45000 [==============================] - 17s 382us/step - loss: 0.0281 - acc: 0.9944 - val_loss: 0.1151 - val_acc: 0.9612
Epoch 2/2
45000/45000 [==============================] - 13s 294us/step - loss: 0.0196 - acc: 0.9976 - val_loss: 0.1116 - val_acc: 0.9616
Q 31+7    T 38   ☑ 38  
Q 88+84   T 172  ☑ 172 
Q 66-619  T -553 ☑ -553
Q 8+243   T 251  ☑ 251 
Q 7-525   T -518 ☑ -518
Q 575-22  T 553  ☑ 553 
Q 45-97   T -52  ☑ -52 
Q 379+756 T 1135 ☑ 1135
Q 911+46  T 957  ☑ 957 
Q 6-73    T -67  ☑ -

# Testing

In [15]:
print("MSG : Prediction")
print("-" * 50)
right = 0
preds = model.predict_classes(test_x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

MSG : Prediction
--------------------------------------------------
Q 428+272 T 700  ☒ 790 
Q 505+65  T 570  ☒ 560 
Q 6-414   T -408 ☑ -408
Q 387-94  T 293  ☑ 293 
Q 622+16  T 638  ☑ 638 
Q 955-418 T 537  ☒ 547 
Q 400-309 T 91   ☒ 190 
Q 457-8   T 449  ☑ 449 
Q 60+639  T 699  ☒ 799 
Q 813+525 T 1338 ☑ 1338
Q 662+4   T 666  ☑ 666 
Q 7+926   T 933  ☑ 933 
Q 333-29  T 304  ☒ 204 
Q 808+730 T 1538 ☑ 1538
Q 805-8   T 797  ☑ 797 
Q 88-114  T -26  ☑ -26 
Q 509+172 T 681  ☑ 681 
Q 8+901   T 909  ☑ 909 
Q 998+79  T 1077 ☒ 1067
Q 3-220   T -217 ☑ -217
Q 388+33  T 421  ☑ 421 
Q 422+66  T 488  ☑ 488 
Q 92-153  T -61  ☑ -61 
Q 70+33   T 103  ☑ 103 
Q 38+971  T 1009 ☒ 1019
Q 49-4    T 45   ☑ 45  
Q 117-613 T -496 ☑ -496
Q 2-926   T -924 ☑ -924
Q 813+2   T 815  ☑ 815 
Q 9-7     T 2    ☑ 2   
Q 18+74   T 92   ☑ 92  
Q 501-44  T 457  ☑ 457 
Q 308+44  T 352  ☑ 352 
Q 7+14    T 21   ☑ 21  
Q 65-9    T 56   ☑ 56  
Q 3-685   T -682 ☑ -682
Q 61+29   T 90   ☑ 90  
Q 776-551 T 225  ☑ 225 
Q 761+273 T 1034 ☒ 1

Q 62+109  T 171  ☑ 171 
Q 593-8   T 585  ☑ 585 
Q 4+765   T 769  ☑ 769 
Q 41+641  T 682  ☑ 682 
Q 9-836   T -827 ☑ -827
Q 8+212   T 220  ☑ 220 
Q 749-694 T 55   ☑ 55  
Q 161-84  T 77   ☑ 77  
Q 773-408 T 365  ☒ 366 
Q 645-75  T 570  ☒ 560 
Q 93-87   T 6    ☒ 5   
Q 50-28   T 22   ☑ 22  
Q 12-734  T -722 ☑ -722
Q 22-89   T -67  ☑ -67 
Q 4+831   T 835  ☑ 835 
Q 752+83  T 835  ☑ 835 
Q 513-772 T -259 ☑ -259
Q 68+81   T 149  ☑ 149 
Q 53+13   T 66   ☑ 66  
Q 35-265  T -230 ☑ -230
Q 15+102  T 117  ☑ 117 
Q 864-1   T 863  ☑ 863 
Q 598-8   T 590  ☑ 590 
Q 668-93  T 575  ☑ 575 
Q 929+379 T 1308 ☒ 1218
Q 81-3    T 78   ☑ 78  
Q 13+17   T 30   ☑ 30  
Q 290+54  T 344  ☑ 344 
Q 79+23   T 102  ☑ 102 
Q 0-419   T -419 ☑ -419
Q 702-913 T -211 ☒ -212
Q 29+435  T 464  ☑ 464 
Q 98+34   T 132  ☑ 132 
Q 5-96    T -91  ☑ -91 
Q 937-861 T 76   ☒ 15  
Q 806+84  T 890  ☑ 890 
Q 66+202  T 268  ☑ 268 
Q 66-316  T -250 ☑ -250
Q 45-899  T -854 ☑ -854
Q 14+1    T 15   ☑ 15  
Q 2+589   T 591  ☑ 591 
Q 60-113  T -53 

Q 59-84   T -25  ☑ -25 
Q 5+14    T 19   ☑ 19  
Q 293-52  T 241  ☑ 241 
Q 57-212  T -155 ☒ -156
Q 907-23  T 884  ☑ 884 
Q 316-1   T 315  ☑ 315 
Q 35-636  T -601 ☑ -601
Q 183-730 T -547 ☒ -557
Q 49+55   T 104  ☑ 104 
Q 429-3   T 426  ☑ 426 
Q 463+7   T 470  ☑ 470 
Q 0-62    T -62  ☑ -62 
Q 373-19  T 354  ☑ 354 
Q 525-4   T 521  ☑ 521 
Q 7-188   T -181 ☑ -181
Q 38-1    T 37   ☑ 37  
Q 18-11   T 7    ☑ 7   
Q 513-865 T -352 ☑ -352
Q 1+988   T 989  ☑ 989 
Q 6+730   T 736  ☑ 736 
Q 553+927 T 1480 ☒ 1470
Q 943-78  T 865  ☑ 865 
Q 71+62   T 133  ☑ 133 
Q 8+670   T 678  ☑ 678 
Q 337+18  T 355  ☑ 355 
Q 878+949 T 1827 ☑ 1827
Q 897+5   T 902  ☑ 902 
Q 56+63   T 119  ☑ 119 
Q 36-564  T -528 ☑ -528
Q 11+61   T 72   ☑ 72  
Q 33-90   T -57  ☑ -57 
Q 993+3   T 996  ☑ 996 
Q 208+783 T 991  ☒ 998 
Q 4-84    T -80  ☑ -80 
Q 3+37    T 40   ☑ 40  
Q 369+2   T 371  ☑ 371 
Q 31+564  T 595  ☒ 695 
Q 588+520 T 1108 ☑ 1108
Q 62+6    T 68   ☑ 68  
Q 26+670  T 696  ☑ 696 
Q 952-39  T 913  ☑ 913 
Q 597-80  T 517 

Q 24+552  T 576  ☑ 576 
Q 43+28   T 71   ☑ 71  
Q 471-99  T 372  ☑ 372 
Q 174+79  T 253  ☑ 253 
Q 775-307 T 468  ☑ 468 
Q 612-540 T 72   ☒ 1   
Q 87+477  T 564  ☑ 564 
Q 4-387   T -383 ☑ -383
Q 992-361 T 631  ☑ 631 
Q 626+492 T 1118 ☑ 1118
Q 695+16  T 711  ☑ 711 
Q 12-978  T -966 ☑ -966
Q 1+584   T 585  ☑ 585 
Q 704+6   T 710  ☑ 710 
Q 620-728 T -108 ☑ -108
Q 562-31  T 531  ☑ 531 
Q 424-25  T 399  ☒ 309 
Q 939+645 T 1584 ☑ 1584
Q 834+6   T 840  ☑ 840 
Q 544-531 T 13   ☒ 1   
Q 135+186 T 321  ☒ 311 
Q 29-749  T -720 ☑ -720
Q 449-39  T 410  ☒ 400 
Q 7+768   T 775  ☑ 775 
Q 247+214 T 461  ☒ 451 
Q 80+577  T 657  ☑ 657 
Q 311-965 T -654 ☑ -654
Q 5-200   T -195 ☑ -195
Q 761+29  T 790  ☑ 790 
Q 56-32   T 24   ☑ 24  
Q 70+22   T 92   ☒ 93  
Q 268-69  T 199  ☑ 199 
Q 32-901  T -869 ☑ -869
Q 509-64  T 445  ☒ 455 
Q 1-363   T -362 ☑ -362
Q 45+541  T 586  ☒ 576 
Q 640+1   T 641  ☑ 641 
Q 51+576  T 627  ☑ 627 
Q 7-321   T -314 ☑ -314
Q 59-51   T 8    ☑ 8   
Q 503-9   T 494  ☑ 494 
Q 123-735 T -612

Q 980-2   T 978  ☑ 978 
Q 209+549 T 758  ☒ 778 
Q 386-298 T 88   ☒ 10  
Q 69-80   T -11  ☒ -21 
Q 237-451 T -214 ☑ -214
Q 27+6    T 33   ☑ 33  
Q 50+475  T 525  ☑ 525 
Q 855+315 T 1170 ☒ 1160
Q 995-16  T 979  ☑ 979 
Q 23+87   T 110  ☑ 110 
Q 20+67   T 87   ☑ 87  
Q 23-641  T -618 ☑ -618
Q 74-965  T -891 ☑ -891
Q 735-633 T 102  ☑ 102 
Q 747-94  T 653  ☑ 653 
Q 35+602  T 637  ☑ 637 
Q 509-0   T 509  ☑ 509 
Q 937-5   T 932  ☑ 932 
Q 554-644 T -90  ☒ -99 
Q 39-852  T -813 ☑ -813
Q 28-71   T -43  ☑ -43 
Q 32+88   T 120  ☑ 120 
Q 74-47   T 27   ☑ 27  
Q 232+287 T 519  ☒ 529 
Q 44-315  T -271 ☑ -271
Q 49+364  T 413  ☑ 413 
Q 59-96   T -37  ☑ -37 
Q 25+399  T 424  ☑ 424 
Q 326-9   T 317  ☑ 317 
Q 312-405 T -93  ☒ -72 
Q 308+95  T 403  ☑ 403 
Q 4-668   T -664 ☑ -664
Q 39+42   T 81   ☑ 81  
Q 312+74  T 386  ☑ 386 
Q 12+21   T 33   ☑ 33  
Q 877+73  T 950  ☑ 950 
Q 48+236  T 284  ☒ 274 
Q 908+9   T 917  ☑ 917 
Q 42+40   T 82   ☑ 82  
Q 37-759  T -722 ☑ -722
Q 443-11  T 432  ☑ 432 
Q 629-56  T 573 

...

Q 957-8   T 949  ☒ 959 
Q 142+7   T 149  ☑ 149 
Q 409+803 T 1212 ☒ 1202
Q 90-674  T -584 ☑ -584
Q 134-83  T 51   ☑ 51  
Q 149+8   T 157  ☑ 157 
Q 6+32    T 38   ☑ 38  
Q 41-728  T -687 ☑ -687
Q 756+796 T 1552 ☑ 1552
Q 74-730  T -656 ☑ -656
Q 7+358   T 365  ☑ 365 
Q 94+466  T 560  ☑ 560 
Q 26+129  T 155  ☑ 155 
Q 86+465  T 551  ☑ 551 
Q 331-905 T -574 ☑ -574
Q 4+996   T 1000 ☒ 190 
Q 68-832  T -764 ☑ -764
Q 670-16  T 654  ☑ 654 
Q 500-409 T 91   ☒ 100 
Q 603-2   T 601  ☑ 601 
Q 4+420   T 424  ☑ 424 
Q 332-736 T -404 ☒ -414
Q 608+11  T 619  ☑ 619 
Q 6+668   T 674  ☑ 674 
Q 3-545   T -542 ☑ -542
Q 3-566   T -563 ☑ -563
Q 819-65  T 754  ☑ 754 
Q 848+78  T 926  ☑ 926 
Q 851+492 T 1343 ☒ 1353
Q 263+44  T 307  ☑ 307 
Q 86-392  T -306 ☑ -306
Q 632-91  T 541  ☑ 541 
Q 505+572 T 1077 ☑ 1077
Q 737+23  T 760  ☒ 750 
Q 22+4    T 26   ☑ 26  
Q 881-515 T 366  ☑ 366 
Q 523+587 T 1110 ☒ 1100
Q 2+596   T 598  ☑ 598 
Q 52+402  T 454  ☑ 454 
Q 82+943  T 1025 ☑ 1025
Q 997-2   T 995  ☑ 995 
Q 973+23  T 996 

Q 27+36   T 63   ☑ 63  
Q 70+88   T 158  ☑ 158 
Q 529-7   T 522  ☑ 522 
Q 60+145  T 205  ☑ 205 
Q 19-592  T -573 ☑ -573
Q 0+695   T 695  ☑ 695 
Q 5-841   T -836 ☑ -836
Q 776+82  T 858  ☑ 858 
Q 750-0   T 750  ☑ 750 
Q 2-571   T -569 ☑ -569
Q 795-14  T 781  ☑ 781 
Q 954-117 T 837  ☒ 847 
Q 365-410 T -45  ☒ -55 
Q 7+548   T 555  ☑ 555 
Q 292+14  T 306  ☑ 306 
Q 56+76   T 132  ☑ 132 
Q 67-766  T -699 ☑ -699
Q 864-875 T -11  ☒ -10 
Q 787-480 T 307  ☒ 216 
Q 974-369 T 605  ☑ 605 
Q 88+78   T 166  ☑ 166 
Q 42-661  T -619 ☑ -619
Q 43+429  T 472  ☑ 472 
Q 363-633 T -270 ☒ -260
Q 33-683  T -650 ☑ -650
Q 4-249   T -245 ☑ -245
Q 36+30   T 66   ☑ 66  
Q 513+39  T 552  ☑ 552 
Q 8+910   T 918  ☑ 918 
Q 165+469 T 634  ☑ 634 
Q 328-475 T -147 ☒ -157
Q 13+445  T 458  ☑ 458 
Q 7+464   T 471  ☑ 471 
Q 63+60   T 123  ☑ 123 
Q 4-645   T -641 ☑ -641
Q 38+8    T 46   ☑ 46  
Q 4-239   T -235 ☑ -235
Q 50-381  T -331 ☑ -331
Q 92-27   T 65   ☑ 65  
Q 110-508 T -398 ☒ -489
Q 590-19  T 571  ☑ 571 
Q 5+832   T 837 

Q 0+38    T 38   ☑ 38  
Q 668-8   T 660  ☑ 660 
Q 93-40   T 53   ☑ 53  
Q 696-90  T 606  ☒ 696 
Q 180-27  T 153  ☑ 153 
Q 91-86   T 5    ☑ 5   
Q 79-76   T 3    ☑ 3   
Q 775+2   T 777  ☑ 777 
Q 578+439 T 1017 ☒ 1006
Q 969+2   T 971  ☑ 971 
Q 1+591   T 592  ☑ 592 
Q 24+0    T 24   ☑ 24  
Q 736-935 T -199 ☒ -200
Q 220+40  T 260  ☑ 260 
Q 59-604  T -545 ☑ -545
Q 6-168   T -162 ☑ -162
Q 347-53  T 294  ☑ 294 
Q 28-16   T 12   ☑ 12  
Q 298+394 T 692  ☒ 682 
Q 852+56  T 908  ☑ 908 
Q 970+328 T 1298 ☒ 1398
Q 641-1   T 640  ☑ 640 
Q 220+560 T 780  ☒ 781 
Q 46+70   T 116  ☑ 116 
Q 23+893  T 916  ☑ 916 
Q 71+412  T 483  ☑ 483 
Q 39+531  T 570  ☒ 560 
Q 25+2    T 27   ☑ 27  
Q 64-40   T 24   ☑ 24  
Q 383-1   T 382  ☑ 382 
Q 68-6    T 62   ☑ 62  
Q 2+478   T 480  ☑ 480 
Q 834+11  T 845  ☑ 845 
Q 155+92  T 247  ☑ 247 
Q 399-92  T 307  ☑ 307 
Q 958+8   T 966  ☑ 966 
Q 302+270 T 572  ☒ 573 
Q 7+932   T 939  ☑ 939 
Q 467-487 T -20  ☒ -10 
Q 6+680   T 686  ☑ 686 
Q 422-9   T 413  ☑ 413 
Q 69-6    T 63  

Q 42-512  T -470 ☑ -470
Q 359+814 T 1173 ☑ 1173
Q 3-618   T -615 ☑ -615
Q 969+97  T 1066 ☑ 1066
Q 520-8   T 512  ☑ 512 
Q 663+24  T 687  ☑ 687 
Q 52-211  T -159 ☑ -159
Q 24-64   T -40  ☑ -40 
Q 742+8   T 750  ☑ 750 
Q 997+9   T 1006 ☑ 1006
Q 13+56   T 69   ☑ 69  
Q 36-796  T -760 ☑ -760
Q 20+94   T 114  ☑ 114 
Q 5-122   T -117 ☑ -117
Q 13-767  T -754 ☑ -754
Q 2-775   T -773 ☑ -773
Q 928-88  T 840  ☒ 830 
Q 91+13   T 104  ☑ 104 
Q 99+26   T 125  ☑ 125 
Q 27-963  T -936 ☑ -936
Q 990-51  T 939  ☒ 949 
Q 959+18  T 977  ☑ 977 
Q 59-562  T -503 ☑ -503
Q 985+8   T 993  ☑ 993 
Q 76-5    T 71   ☑ 71  
Q 23+68   T 91   ☑ 91  
Q 115-73  T 42   ☑ 42  
Q 230-99  T 131  ☑ 131 
Q 66+20   T 86   ☑ 86  
Q 558-52  T 506  ☑ 506 
Q 925+87  T 1012 ☑ 1012
Q 39-520  T -481 ☑ -481
Q 89-990  T -901 ☑ -901
Q 68-66   T 2    ☑ 2   
Q 71-82   T -11  ☑ -11 
Q 336+74  T 410  ☒ 400 
Q 26-375  T -349 ☑ -349
Q 13-244  T -231 ☑ -231
Q 174-60  T 114  ☑ 114 
Q 561+3   T 564  ☑ 564 
Q 775+33  T 808  ☑ 808 
Q 14-57   T -43 

Q 6+655   T 661  ☑ 661 
Q 6-993   T -987 ☑ -987
Q 968+176 T 1144 ☑ 1144
Q 281+3   T 284  ☑ 284 
MSG : Accuracy is 0.8675


In [16]:
# print("MSG : Prediction")
# test_x = ["555+275", "860+7  ", "340+29 "]
# test_y = ["830 ", "867 ", "369 "]
# x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
# y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
# for j, (i, c) in enumerate(zip(test_x, test_y)):
#     x[j] = ctable.encode(i, MAXLEN)
#     y[j] = ctable.encode(c, DIGITS + 1)